In [37]:
class N_gram:
  def __init__(self):
    pass

  def n_gram(self, text, n):
    self.text = text
    # self.text = text.lower()
    # self.text = self.text.split()
    self.n = n
    Ngram = [tuple(self.text[index : index + self.n]) for index in range(0, len(self.text) - self.n + 1)]
    return Ngram

In [38]:
import math
class nist_score(N_gram):
  def __init__(self):
    self.n_gram_order = 5

  def fit(self, refs, predict):
    pred_ngrams = [self.n_gram(predict, n) for n in range(1, self.n_gram_order + 1)]
    refs_ngrams = [[self.n_gram(ref, n) for n in range(1, self.n_gram_order + 1)] for ref in refs]

    total_reference_words = 0
    n_grams_refq = dict()
    for index_refs in range(0, len(refs)):
      total_reference_words += len(refs[index_refs])
      for n in range(0, self.n_gram_order):
        _count_ngrams = self._counter(refs_ngrams[index_refs][n])
        for element in _count_ngrams:
          if element in n_grams_refq:
            n_grams_refq[element] += _count_ngrams[element]
          elif element not in n_grams_refq:
            n_grams_refq[element] = _count_ngrams[element]


    info = dict()
    for W_1_n in n_grams_refq:
      W_1_m = W_1_n[ : -1]
      if W_1_m and W_1_m in n_grams_refq:
        occurrence = n_grams_refq[W_1_m]
      else:
        occurrence = total_reference_words
      info[W_1_n] = math.log(occurrence / n_grams_refq[W_1_n], 2)

    nist_scores = []
    ref_lenght = 0
    pred_lenght = 0
    for n_gram in range(0, self.n_gram_order):

      nist_precisions = []
      for index_refs in range(0, len(refs)):
        pred_ngram_count = self._counter(pred_ngrams[n_gram])
        ref_ngram_count = self._counter(refs_ngrams[index_refs][n_gram])
        overlaps_ngram = self._calculate_overlaps(pred_ngram_count, ref_ngram_count)
        nist_numerator = sum([info[ngram] * count_ngram for ngram , count_ngram in overlaps_ngram.items()])
        nist_denominator = sum(pred_ngram_count.values())
        nist_prnist_scoreecision = None
        if nist_denominator == 0:
          nist_precision = 0
        else:
          nist_precision = nist_numerator / nist_denominator
        nist_precisions.append(nist_precision)
      nist_scores.append(max(nist_precisions))
      ref_lenght += len(refs[nist_precisions.index(max(nist_precisions))])
      pred_lenght += len(predict)
    return sum(nist_scores) * self._length_penalty(ref_lenght, pred_lenght)




  def _calculate_overlaps(self, dict_1, dict_2):
    overlaps = dict()
    for key in dict_1:
      if key in dict_2:
        _val = min(dict_1[key], dict_2[key])
        overlaps.update({key : _val})
    return overlaps

  def _counter(self, temp):
    counter = dict()
    for element in temp:
      if element in counter:
        counter[element] += 1
      elif element not in counter:
        counter[element] = 1
    return counter

  def _length_penalty(self, refs_lenght, pred_lenght):
    ratio = pred_lenght / refs_lenght
    if 0 < ratio < 1:
        ratio_x, score_x = 1.5, 0.5
        beta = math.log(score_x) / math.log(ratio_x) ** 2
        return math.exp(beta * math.log(ratio) ** 2)
    else:  # ratio <= 0 or ratio >= 1
        return max(min(ratio, 1.0), 0.0)

